# Prediction based on probability - Use loan grade

In [1]:
import pandas
import numpy
import matplotlib.pyplot as plt

In [2]:
import loan_approval_lib

In [3]:
data = loan_approval_lib.load_data()

In [4]:
data_train = data['train']

In [5]:
data_test = data['test']

In [6]:
len(data_test)

39098

In [7]:
data_train

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,58640,34,120000,MORTGAGE,5.0,EDUCATION,D,25000,15.95,0.21,Y,10,0
58641,58641,28,28800,RENT,0.0,MEDICAL,C,10000,12.73,0.35,N,8,1
58642,58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2,1
58643,58643,22,30000,RENT,2.0,EDUCATION,A,5000,8.90,0.17,N,3,0


In [8]:
data_test

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4
...,...,...,...,...,...,...,...,...,...,...,...,...
39093,97738,22,31200,MORTGAGE,2.0,DEBTCONSOLIDATION,B,3000,10.37,0.10,N,4
39094,97739,22,48000,MORTGAGE,6.0,EDUCATION,A,7000,6.03,0.15,N,3
39095,97740,51,60000,MORTGAGE,0.0,PERSONAL,A,15000,7.51,0.25,N,25
39096,97741,22,36000,MORTGAGE,4.0,PERSONAL,D,14000,15.62,0.39,Y,4


In [9]:
data_train['loan_grade'].unique()

array(['B', 'C', 'A', 'D', 'E', 'F', 'G'], dtype=object)

In [10]:
model = data_train[['loan_grade', 'loan_status']].groupby(['loan_grade']).aggregate(['mean', 'sum'])

In [11]:
model

loan_status      
                  mean   sum
loan_grade                  
A             0.049180  1032
B             0.102304  2087
C             0.135375  1494
D             0.593564  2988
E             0.625372   631
F             0.610738    91
G             0.818182    27

In [12]:
model.index

Index(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='object', name='loan_grade')

In [13]:
model.columns

MultiIndex([('loan_status', 'mean'),
            ('loan_status',  'sum')],
           )

In [14]:
model['loan_status']['mean']

loan_grade
A    0.049180
B    0.102304
C    0.135375
D    0.593564
E    0.625372
F    0.610738
G    0.818182
Name: mean, dtype: float64

In [15]:
model_dict = {
    key: value for (key, value) in model['loan_status']['mean'].items()
}

In [16]:
model_dict

{'A': 0.04918032786885246,
 'B': 0.10230392156862746,
 'C': 0.13537513591881117,
 'D': 0.5935637663885578,
 'E': 0.6253716551040635,
 'F': 0.610738255033557,
 'G': 0.8181818181818182}

### Generate Training Output

In [19]:
data_train['loan_status_predict'] = list(map(lambda x: round(model_dict[x]), data_train['loan_grade']))

In [20]:
select_query = data_train['loan_status'] == data_train['loan_status_predict']
P = len(data_train[select_query]) / len(data_train)
P

0.8789155085685054

### Generate Output

In [23]:
data_test['loan_status'] = list(map(lambda x: round(model_dict[x]), data_test['loan_grade']))

In [24]:
data_test_submission = data_test[['id', 'loan_status']]

In [25]:
data_test_submission

,id,loan_status
0,58645,1
1,58646,0
2,58647,1
3,58648,0
4,58649,1
...,...,...
39093,97738,0
39094,97739,0
39095,97740,0
39096,97741,1


In [26]:
data_test_submission.to_csv('data_test_submission_4.csv', index=False)